# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. Klein  ->  R. Klein  |  ['R. Klein']
M. Flock  ->  M. Flock  |  ['M. Flock']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 68 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.07785


extracting tarball to tmp_2305.07785...

 done.
Retrieving document from  https://arxiv.org/e-print/2305.07864


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2305.07785/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

✔ → 103924:\section{Numerical Experiments}
  ↳ 150815:\section{Horizontal Convection}


✔ → 150815:\section{Horizontal Convection}
  ↳ 160011:\section{Conclusion}
✔ → 160011:\section{Conclusion}
  ↳ 167776:\section{Movies}


✔ → 167776:\section{Movies}
  ↳ 172298:end


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure mod2D256T1.VMAXR
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure mod2D256T1.VMAXR as tmp_2305.08165/./mod2D256T1.VMAXR.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure mod2D256T1.VMAXR
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure mod2D256T1.VMAXR as tmp_2305.08165/./mod2D256T1.VMAXR.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/tmp/ipykernel_2086/1863104449.py:49: LatexWarning: 2305.08165 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.07785-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.07785) | **Unveiling the formation of the massive DR21 ridge**  |
|| L. Bonne, et al. -- incl., <mark>R. Klein</mark> |
|*Appeared on*| *2023-05-16*|
|*Comments*| *32 pages, 28 figures, accepted in ApJ*|
|**Abstract**| We present new $^{13}$CO(1-0), C$^{18}$O(1-0), HCO$^{+}$(1-0) and H$^{13}$CO$^{+}$(1-0) maps from the IRAM 30m telescope, and a spectrally-resolved [CII] 158 $\mu$m map observed with the SOFIA telescope towards the massive DR21 cloud. This traces the kinematics from low- to high-density gas in the cloud which allows to constrain the formation scenario of the high-mass star forming DR21 ridge. The molecular line data reveals that the sub-filaments are systematically redshifted relative to the dense ridge. We demonstrate that [CII] unveils the surrounding CO-poor gas of the dense filaments in the DR21 cloud. We also show that this surrounding gas is organized in a flattened cloud with curved redshifted dynamics perpendicular to the ridge. The sub-filaments thus form in this curved and flattened mass reservoir. A virial analysis of the different lines indicates that self-gravity should drive the evolution of the ridge and surrounding cloud. Combining all results we propose that bending of the magnetic field, due to the interaction with a mostly atomic colliding cloud, explains the velocity field and resulting mass accretion on the ridge. This is remarkably similar to what was found for at least two nearby low-mass filaments. We tentatively propose that this scenario might be a widespread mechanism to initiate star formation in the Milky Way. However, in contrast to low-mass clouds, gravitational collapse plays a role on the pc scale of the DR21 ridge because of the higher density. This allows more effective mass collection at the centers of collapse and should facilitate massive cluster formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.07864-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.07864) | **Meridional Circulation driven by Planetary Spiral Wakes in Radiative and  Magnetized Protoplanetary Discs**  |
|| M. Cilibrasi, <mark>M. Flock</mark>, J. Szulágyi |
|*Appeared on*| *2023-05-16*|
|*Comments*| **|
|**Abstract**| We study a Jupiter-mass planet formation for the first time in radiative magneto-hydrodynamics (MHD) simulations and compare it with pure hydrodynamical simulations, as well as to different isothermal configurations. We found that the meridional circulation is the same in every setup. The planetary spiral wakes drive a vertical stirring inside the protoplanetary disc and the encounter with these shock fronts also helps in delivering gas vertically onto the Hill-sphere. The accretion dynamics are unchanged: the planet accretes vertically, and there is outflow in the midplane regions inside the Hill-sphere. We determined the effective $\alpha$-viscosity generated in the disc by the various angular momentum loss mechanisms, which showed that magnetic fields produce high turbulence in the ideal MHD limit, that grows from $\alpha \sim 10^{-2.5}$ up to $\sim 10^{-1.5}$ after the planet spirals develop. In the HD simulations, the planetary spirals contribute to $\alpha \sim 10^{-3}$, making this a very important angular momentum transport mechanism. Due to the various $\alpha$ values in the different setups, the gap opening is different in each case. In the radiative MHD setups, the high turbulent viscosity prevents gap opening, leading to a higher Hill mass, and no clear dust trapping regions. While the Hill accretion rate is $10^{-6} \rm{M_{Jup}/yr}$ in all setups, the accretion variability is orders of magnitude higher in radiative runs than in isothermal ones. Finally, with higher-resolution runs, the magneto-rotational instability started to be resolved, changing the effective viscosity and increasing the heating in the disc. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.08165-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.08165) | **Thermal instabilities in accretion disks II: Numerical Experiments for  the Goldreich-Schubert-Fricke Instability and the Convective Overstability in  disks around young stars**  |
|| <mark>H. Klahr</mark>, H. Baehr, J. D. M. Fuksman |
|*Appeared on*| *2023-05-16*|
|*Comments*| *ApJ, in press: 27 pages, 18 figures, 4 Movies*|
|**Abstract**| The linear stability analysis of a stratified rotating fluid (see paper I) showed that disks with a baroclinic stratification under the influence of thermal relaxation will become unstable to thermal instabilities. One instability is the Goldreich-Schubert-Fricke instability (GSF), which is the local version of the Vertical Shear Instability (VSI) and the other is a thermal overstability, the Convective Overstability (COS). In the present paper we reproduce the analytic predicted growth rates for both instabilities in numerical experiments of small axisymmetric sections of vertically isothermal disks with a radial temperature gradient, especially for cooling times longer than the critical cooling time for VSI. In this cooling time regime our simulations reveal the simultaneous and independent growth of both modes: COS and GSF. We consistently observe that GSF modes exhibit a faster growth rate compared to COS modes. Near the midplane, GSF modes eventually stop growing, while COS modes continue to grow and ultimately dominate the flow pattern. Away from the midplane, we find GSF modes to saturate, when bands of constant angular momentum have formed. In these bands we observe the formation and growth of eddies driven by the baroclinic term, further enhancing the velocity perturbations. In geophysics this effect is known as horizontal convection or sea-breeze instability. Three-dimensional simulations will have to show whether similar effects will occur when axisymmetry is not enforced. Our local simulations help to reveal the numerical resolution requirements to observe thermal instabilities in global simulations of disks around young stars. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

207  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
